In [1]:
import string
import re
import os
import sys
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

from utils import *
from models import *
import yaml
import numpy as np

In [49]:
class Options:
    def __init__(self):
        self.model_name = 'alp'

options = Options()
options.output_file = 'test_output/reapos_test_text.txt'
options.params_file = 'params_joke.yaml'
options.use_cuda = False
options.use_validation = True
options.gpu2cpu = True

In [50]:
#LOAD CONFIGURATIONS AND LANGUAGES
USE_CUDA = options.use_cuda
print("Use cuda: %s" %USE_CUDA)

try:
    with open(options.params_file, 'r') as ymlfile:
        config = yaml.load(ymlfile)
except:
    sys.exit("Parameters file missing")

#Setup languages
INPUT_LANG_CODE = config['INPUT_LANG']
OUTPUT_LANG_CODE = config['OUTPUT_LANG']

INPUT_LANG_PUNC_LEVEL = config["INPUT_LANG_PUNC_LEVEL"]
OUTPUT_LANG_PUNC_LEVEL = config["OUTPUT_LANG_PUNC_LEVEL"]

if INPUT_LANG_CODE == 'en' and OUTPUT_LANG_CODE == 'es':
    lang_en = input_lang = Lang(INPUT_LANG_CODE, config["W2V_EN_PATH"], config["DICT_EN_PATH"], punctuation_level=INPUT_LANG_PUNC_LEVEL)
    lang_es = output_lang = Lang(OUTPUT_LANG_CODE, config["W2V_ES_PATH"], config["DICT_ES_PATH"], punctuation_level=OUTPUT_LANG_PUNC_LEVEL)
elif INPUT_LANG_CODE == 'es' and OUTPUT_LANG_CODE == 'en':
    lang_es = input_lang = Lang(INPUT_LANG_CODE, config["W2V_ES_PATH"], config["DICT_ES_PATH"], punctuation_level=INPUT_LANG_PUNC_LEVEL)
    lang_en = output_lang = Lang(OUTPUT_LANG_CODE, config["W2V_EN_PATH"], config["DICT_EN_PATH"], punctuation_level=OUTPUT_LANG_PUNC_LEVEL)

input_prosody_params = config['INPUT_PROSODY']
if input_prosody_params == None:
    input_prosody_params = []
output_prosody_params = config['OUTPUT_PROSODY']
if output_prosody_params == None:
    output_prosody_params = []    
    
#NETWORK CONFIG
max_seq_length = int(config['MAX_SEQ_LENGTH'])
n_prosody_params = int(config['N_PROSODY_PARAMS'])
input_prosody_params = config['INPUT_PROSODY']
encoder_type = config['ENCODER_TYPE']
attn_model = config['ATTN_MODEL']
hidden_size = int(config['HIDDEN_SIZE'])
n_layers = int(config['N_LAYERS'])

2018-08-13 13:10:34,174 : INFO : loading Word2Vec object from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_en_heroes.model


Use cuda: False


2018-08-13 13:10:36,406 : INFO : loading wv recursively from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_en_heroes.model.wv.* with mmap=None
2018-08-13 13:10:36,407 : INFO : loading vectors from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_en_heroes.model.wv.vectors.npy with mmap=None
2018-08-13 13:10:36,533 : INFO : setting ignored attribute vectors_norm to None
2018-08-13 13:10:36,537 : INFO : loading vocabulary recursively from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_en_heroes.model.vocabulary.* with mmap=None
2018-08-13 13:10:36,540 : INFO : loading trainables recursively from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_en_heroes.model.trainables.* with mmap=None
2018-08-13 13:10:36,543 : INFO : loading syn1neg from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_en_heroes.model.trainables.syn1neg.npy with mmap=None
2018-08-13 13:10:36

en Vocabulary size: 30000


2018-08-13 13:10:39,614 : INFO : loading wv recursively from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_es_heroes.model.wv.* with mmap=None
2018-08-13 13:10:39,615 : INFO : loading vectors from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_es_heroes.model.wv.vectors.npy with mmap=None
2018-08-13 13:10:39,826 : INFO : setting ignored attribute vectors_norm to None
2018-08-13 13:10:39,841 : INFO : loading vocabulary recursively from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_es_heroes.model.vocabulary.* with mmap=None
2018-08-13 13:10:39,856 : INFO : loading trainables recursively from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_es_heroes.model.trainables.* with mmap=None
2018-08-13 13:10:39,861 : INFO : loading syn1neg from /Users/alp/Documents/Corpora/OpenSubtitles2018/w2v/reapos_min5_heroes/w2v_es_heroes.model.trainables.syn1neg.npy with mmap=None
2018-08-13 13:10:40

es Vocabulary size: 30000


In [51]:
#LOAD DATASETS    
AUDIO_TEST_DATA_PATH = config["AUDIO_TEST_DATA_FILE"]
AUDIO_TRAIN_DATA_PATH = config["AUDIO_TRAIN_DATA_FILE"]
AUDIO_VALIDATION_DATA_PATH = config["AUDIO_VALIDATION_DATA_FILE"]
TEXT_TEST_DATA_PATH = config['TEXT_TEST_DATA_PATH']

In [13]:
#Initialize text models
text_encoder_path = 'models/5mmheroes_unpuncdinput_encoder.model'
text_decoder_path = 'models/5mmheroes_unpuncdinput_decoder.model'

print('Input punc lvl: ', INPUT_LANG_PUNC_LEVEL)
print('Output punc lvl: ', OUTPUT_LANG_PUNC_LEVEL)

text_encoder = GenericEncoder(input_lang.vocabulary_size, hidden_size, input_lang.get_weights_matrix(), n_layers)
text_decoder = LuongAttnDecoderRNN(attn_model, hidden_size, output_lang.get_weights_matrix(), output_lang.vocabulary_size, n_layers, input_feed=config['DECODER_INPUT_FEED'])
load_model(text_encoder, text_decoder, text_encoder_path, text_decoder_path, options.gpu2cpu)

Input punc lvl:  0
Output punc lvl:  2
gpu2cpu: True


In [61]:
#Initialize prosodic models
AUDIO_ENCODE_ONLY = False
model_name = "audio_pausein_dummyout_miniset"
prosodic_encoder_path = 'models/' + model_name + '_encoder.model'
prosodic_decoder_path = 'models/' + model_name + '_decoder.model'

if encoder_type == 'sum':
    prosodic_encoder = EncoderRNN_sum(input_lang.vocabulary_size, n_prosody_params, hidden_size, input_lang.get_weights_matrix(), n_layers)
elif encoder_type == 'parallel':
    prosodic_encoder = EncoderRNN_parallel(input_lang.vocabulary_size, n_prosody_params, hidden_size, input_lang.get_weights_matrix(), n_layers)
else:
    sys.exit("Unrecognized encoder type. Check params file. Exiting...")
if AUDIO_ENCODE_ONLY:
    prosodic_decoder = LuongAttnDecoderRNN(attn_model, hidden_size, output_lang.get_weights_matrix(), output_lang.vocabulary_size, n_layers, USE_CUDA=USE_CUDA)
else:
    prosodic_decoder = ProsodicDecoderRNN(attn_model, hidden_size, output_lang.get_weights_matrix(), output_lang.vocabulary_size, n_layers, USE_CUDA=USE_CUDA)
really_prosodic_decoder = ProsodicDecoderRNN(attn_model, hidden_size, output_lang.get_weights_matrix(), output_lang.vocabulary_size, n_layers, USE_CUDA=USE_CUDA)
    
load_model(prosodic_encoder, prosodic_decoder, prosodic_encoder_path, prosodic_decoder_path, gpu_to_cpu=options.gpu2cpu)

gpu2cpu: True


In [ ]:
print(prosodic_encoder.state_dict)

In [14]:
#DATA GENERATORS
def text_data_generator(data_path, input_lang, output_lang, stop_at=-1):
    count = 0
    with open(data_path,'r') as inputfile:
        for line in inputfile:
            if not stop_at == -1 and count >= stop_at:
                break
            pair = [sentence.strip() for sentence in line.split('\t')]
            if input_lang.lang_code == 'en':
                in_sentence = pair[0]
                out_sentence = pair[1]
            elif input_lang.lang_code == 'es':
                in_sentence = pair[1]
                out_sentence = pair[0]

            in_sentence_tokens = in_sentence.lower().split()
            out_sentence_tokens = out_sentence.lower().split()

            if input_lang.punctuation_level == 0:
                in_sentence_tokens = remove_punc_tokens(in_sentence_tokens)
            elif input_lang.punctuation_level == 1:
                in_sentence_tokens = remove_punc_tokens(in_sentence_tokens, keep_main_puncs=True)
            if output_lang.punctuation_level == 0:
                out_sentence_tokens = remove_punc_tokens(out_sentence_tokens)
            elif output_lang.punctuation_level == 1:
                out_sentence_tokens = remove_punc_tokens(out_sentence_tokens, keep_main_puncs=True)

            count += 1

            yield in_sentence_tokens, out_sentence_tokens
            

def audio_data_generator(data_path, input_lang, output_lang, n_prosody_params, input_prosody_params, output_prosody_params, dummyfy_input_prosody=False, dummyfy_output_prosody=False, stop_at = -1):
    assert not input_lang == output_lang
    audio_data = read_audio_dataset_file(data_path, shuffle=False)

    #start generating samples from the proscript links in the data file
    count = 0
    for segment_data in audio_data:
        if not stop_at == -1 and count >= stop_at:
            break

        es_txt = segment_data[0]
        es_csv = segment_data[1]
        en_txt = segment_data[2]
        en_csv = segment_data[3]
        
        #print(en_csv)
        #print(es_csv)

        if input_lang.lang_code == 'en' and output_lang.lang_code == 'es':
            input_proscript = en_csv
            output_proscript = es_csv
            #input_transcript = read_text_file(en_txt)
            #output_transcript = read_text_file(es_txt)
        elif input_lang.lang_code == 'es' and output_lang.lang_code == 'en':
            input_proscript = es_csv
            output_proscript = en_csv
            #input_transcript = read_text_file(es_txt)
            #output_transcript = read_text_file(en_txt)
            
        if input_lang.punctuation_level == 0:
            input_punc = False
            input_only_main_punc = False
        elif input_lang.punctuation_level == 1:
            input_punc = True
            input_only_main_punc = True
        elif input_lang.punctuation_level == 2:
            input_punc = True
            input_only_main_punc = False
            
        if output_lang.punctuation_level == 0:
            output_punc = False
            output_only_main_punc = False
        elif output_lang.punctuation_level == 1:
            output_punc = True
            output_only_main_punc = True
        elif output_lang.punctuation_level == 2:
            output_punc = True
            output_only_main_punc = False

        in_sentence_tokens, in_prosody_tokens = read_data_from_proscript(input_proscript, input_lang, n_prosody_params, input_prosody_params, punctuation_as_tokens = input_punc, keep_only_main_puncs = input_only_main_punc)
        out_sentence_tokens, out_prosody_tokens = read_data_from_proscript(output_proscript, output_lang, n_prosody_params, output_prosody_params, punctuation_as_tokens = output_punc, keep_only_main_puncs = output_only_main_punc)
    
        if dummyfy_input_prosody:
            in_prosody_tokens = np.zeros_like(in_prosody_tokens)
        if dummyfy_output_prosody:
            out_prosody_tokens = np.zeros_like(out_prosody_tokens)
            
        count += 1
        yield in_sentence_tokens, in_prosody_tokens, out_sentence_tokens, out_prosody_tokens

In [15]:
#various utilities
def print_prosody(prosody):
    print(np.array(prosody).transpose())
    
def print_tokens_with_pause(tokens, pausevals = [], pauseflags=[]):
    if pauseflags == [] and not pausevals == []:
        pauseflags = flags_from_value(pausevals)
        
    to_print = ""
    for i, token in enumerate(tokens):
        to_print += token + " "
        #if not pausevals == [] and pauseflags[i]:
        if not pausevals == []:
            to_print += "[" + "{:.2f}".format(pausevals[i]) + "]"
            if pauseflags[i]:
                to_print += "[1]"
            to_print += " "
            
    print(to_print)
    return to_print
            
def flags_from_value(prosody_seq):
    return [1 if not feature_value == 0.0 else 0 for feature_value in prosody_seq]

In [ ]:
#SEE TEXT DATA
for in_sent, out_sent in text_data_generator(TEXT_TEST_DATA_PATH, input_lang, output_lang):
    print(in_sent)
    print(out_sent)
    exit = input('...')
    if exit == 'q':
        break

In [ ]:
#SEE AUDIO DATA
in_pros = []
for in_sent, in_pros, out_sent, out_pros in audio_data_generator(AUDIO_TEST_DATA_PATH, input_lang, output_lang, n_prosody_params, input_prosody_params, output_prosody_params, dummyfy_input_prosody=False, dummyfy_output_prosody=False):
    print(in_sent)
    print(indexes_from_tokens(input_lang, in_sent))
    in_pros = finalize_prosody_sequence(in_pros)
    print_prosody(in_pros)
    print(out_sent)
    print_prosody(out_pros)
    exit = input('...')
    if exit == 'q':
        break

In [21]:
#EVALUATORS
#text -> text
def evaluate_text(input_seq_tokens, input_lang, output_lang, encoder, decoder, max_length, USE_CUDA=False):
    input_word_seqs = [indexes_from_tokens(input_lang, input_seq_tokens)]
    
    #make sure sequences are below max_length. 
    input_word_seqs = limit_seqs_to_max(input_word_seqs, max_length)

    input_lengths = [len(input_word_seqs[0])]
    input_word_batch = Variable(torch.LongTensor(input_word_seqs)).transpose(0, 1)

    if USE_CUDA:
        input_word_batch = input_batch.cuda()
        input_prosody_batch = input_prosody_batch.cuda()

    # Set to not-training mode to disable dropout
    encoder.train(False)
    decoder.train(False)

    # Run through encoder
    encoder_outputs, encoder_hidden = encoder(input_word_batch, input_lengths, None)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([output_lang.token2index(SWT_TOKEN)])) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))

    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        decoder_context = decoder_context.cuda()

    # Store output words and attention states
    decoded_words = []
    decoder_attentions = torch.zeros(max_length + 1, max_length + 1)

    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_context, decoder_hidden, decoder_attn = decoder(
                decoder_input, decoder_context, decoder_hidden, encoder_outputs)
        
        decoder_attentions[di,:decoder_attn.size(2)] += decoder_attn.squeeze(0).squeeze(0).cpu().data

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        #ni = topi[0][0]  #old code
        ni = topi.item()
        if ni == output_lang.token2index(EOS_TOKEN):
            decoded_words.append(EOS_TOKEN)
            break
        else:
            decoded_words.append(output_lang.index2token(ni))

        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([ni]))
        if USE_CUDA: decoder_input = decoder_input.cuda()

    # Set back to training mode
    encoder.train(True)
    decoder.train(True)

    return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]

#text+audio -> text+audio
#use prosodic encoder/decoder
def evaluate_audio(input_seq_tokens, input_prosody_tokens, input_lang, output_lang, encoder, decoder, max_length, audio_encode_only=AUDIO_ENCODE_ONLY, USE_CUDA=False):
    input_word_seqs = [indexes_from_tokens(input_lang, input_seq_tokens)]
    input_prosody_seqs = [finalize_prosody_sequence(input_prosody_tokens)] #put the end token
    
    #make sure sequences are below max_length. 
    input_word_seqs = limit_seqs_to_max(input_word_seqs, max_length)
    input_prosody_seqs = limit_seqs_to_max(input_prosody_seqs, max_length)

    input_lengths = [len(input_word_seqs[0])]
    input_word_batch = Variable(torch.LongTensor(input_word_seqs)).transpose(0, 1)
    input_prosody_batch = Variable(torch.FloatTensor(input_prosody_seqs)).transpose(0, 1)
    
    if USE_CUDA:
        input_word_batch = input_batch.cuda()
        input_prosody_batch = input_prosody_batch.cuda()

    # Set to not-training mode to disable dropout
    encoder.train(False)
    decoder.train(False)

    # Run through encoder
    encoder_outputs, encoder_hidden = encoder(input_word_batch, input_prosody_batch, input_lengths, None)

    # Create starting vectors for decoder
    decoder_word_input = Variable(torch.LongTensor([output_lang.token2index(SWT_TOKEN)])) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))

    if USE_CUDA:
        decoder_word_input = decoder_word_input.cuda()
        decoder_context = decoder_context.cuda()

    # Store output words and attention states
    decoded_word_seq = []
    decoded_pauseflag_seq = []
    decoded_pausevalue_seq = []
    decoder_attentions = torch.zeros(max_length + 1, max_length + 1)

    # Run through decoder
    decoder_stop = False
    for di in range(max_length):
        if audio_encode_only:
            decoder_output_word, decoder_context, decoder_hidden, decoder_attention = decoder(
                decoder_word_input, decoder_context, decoder_hidden, encoder_outputs)
        else:
            decoder_output_word, decoder_output_pauseflag, decoder_output_pausevalue, decoder_context, decoder_hidden, decoder_attention = decoder(
                decoder_word_input, decoder_context, decoder_hidden, encoder_outputs
            )
        
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data
            
        # Choose top word from output
        topv, topi = decoder_output_word.data.topk(1)
        ni_word = topi.item()
        if ni_word == output_lang.token2index(EOS_TOKEN):
            decoded_word_seq.append(EOS_TOKEN)
            #decoder_stop = True
        else:
            decoded_word_seq.append(output_lang.index2token(ni_word))
        
        if not audio_encode_only:
            # Look at pauseflag output
            topv, topi = decoder_output_pauseflag.data.topk(1)
            ni_pauseflag = topi.item()
            decoded_pauseflag_seq.append(ni_pauseflag)

            # Look at pauseval output
            predicted_pausevalue = decoder_output_pausevalue.item()
            #decoded_pausevalue_seq.append(unnormalize_value(predicted_pausevalue, 0.0, 10.0))
            decoded_pausevalue_seq.append(predicted_pausevalue)

        # Next input is chosen word
        if decoder_stop:
            break
        else:
            decoder_word_input = Variable(torch.LongTensor([ni_word]))
            if USE_CUDA: decoder_word_input = decoder_word_input.cuda()

    # Set back to training mode
    encoder.train(True)
    decoder.train(True)

    return decoded_word_seq, decoded_pauseflag_seq, decoded_pausevalue_seq, decoder_attentions[:di+1, :len(encoder_outputs)]

In [17]:
#Text -> text evaluator test
input_sentence = "i found the other seven yesterday" 
input_seq_tokens = input_sentence.split()
decoded_words, _ = evaluate_text(input_seq_tokens, input_lang, output_lang, text_encoder, text_decoder, max_seq_length)
print(readable_from_tokens(decoded_words[:-1]))

he encontrado a los otros siete .


In [52]:
gen = audio_data_generator(AUDIO_TRAIN_DATA_PATH, input_lang, output_lang, n_prosody_params, input_prosody_params, output_prosody_params)

In [62]:
#Text+prosody -> text+prosody on audio data visualization
print_every = 1
print_count = 0
for in_sentence_tokens, in_prosody_tokens , out_sentence_tokens, out_prosody_tokens in gen:
    print_count += 1
    print("IN")
    print_tokens_with_pause(in_sentence_tokens, in_prosody_tokens[:,0])
    print("GT")
    print_tokens_with_pause(out_sentence_tokens, out_prosody_tokens[:,0])
    print("OUT PROSODY")
    translation_tokens, pauseflag_tokens, pausevalue_tokens, _ = evaluate_audio(in_sentence_tokens, in_prosody_tokens, input_lang, output_lang, prosodic_encoder, prosodic_decoder, max_seq_length, audio_encode_only=AUDIO_ENCODE_ONLY)
    print_tokens_with_pause(translation_tokens, pausevalue_tokens, pauseflag_tokens)
    print("OUT TEXT")
    translation_tokens, _ = evaluate_text(in_sentence_tokens, input_lang, output_lang, text_encoder, text_decoder, max_seq_length)
    print_tokens_with_pause(translation_tokens)
    
    if print_count % print_every == 0:
        inp = input("...")
        if inp == 'q':
            break
    print("======================================================================")

IN
i [0.00] am [0.18][1] shutting [0.00] this [0.00] operation [0.05][1] down [0.54][1] today [0.38][1] and [0.00] then [0.00] i [0.00] 'm [0.00] going [0.00] to [0.00] the [0.03][1] ag [0.00] 's [0.00] office [0.14][1] and [0.00] i [0.00] 'm [0.00] filing [0.00] human [0.00] rights [0.00] violations [0.00] against [0.00] you [0.00] senator [0.00] 
GT
voy [0.00] a [0.00] cerrar [0.03][1] este [0.03][1] proyecto [0.41][1] hoy [0.20][1] mismo [0.51][1] . [0.00] luego [0.09][1] iré [0.00] al [0.00] fiscal [0.00] general [0.06][1] ... [0.00] y [0.00] le [0.00] demandaré [0.00] por [0.00] violación [0.00] de [0.00] los [0.00] derechos [0.00] humanos [0.00] , [0.00] senador [0.00] . [0.00] 
OUT PROSODY


/Users/alp/extSW/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:6: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/alp/extSW/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


voy [0.04] a [0.04] cerrar [0.04] este [0.04] proyecto [0.04] hoy [0.04] mismo [0.04] . [0.04] luego [0.04] iré [0.04] al [0.04] fiscal [0.04] general [0.04] ... [0.04] y [0.04] le [0.04] demandaré [0.04] por [0.04] violación [0.04] . [0.04] END [0.04] , [0.03] senador [0.03] . [0.04] END [0.03] , [0.03] senador [0.04] . [0.04] END [0.03] , [0.03] senador [0.04] . [0.04] END [0.03] , [0.03] senador [0.04] . [0.04] END [0.03] , [0.03] senador [0.04] . [0.04] 
OUT TEXT
voy a cerrar esta investigación , y luego voy a ir a la oficina de la oficina de la oficina . END 
...
IN
do [0.00] i [0.00] have [0.00] to [0.00] choose [0.00] 
GT
¿ [0.00] tengo [0.00] que [0.00] elegir [0.00] ? [0.00] 
OUT PROSODY
¿ [0.04] tengo [0.04] que [0.04] elegir [0.04] ? [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04] END [0.04] . [0.04

In [ ]:
#Text -> text on text data
def text_set_translation_generator(stop_at = -1, report=False):
    for in_sentence_tokens, out_sentence_tokens in text_data_generator(TEXT_TEST_DATA_PATH, input_lang, output_lang, stop_at):
        translation_tokens, _ = evaluate_text(in_sentence_tokens, input_lang, output_lang, text_encoder, text_decoder, max_seq_length)
        if report:
            print("> %s"%(readable_from_tokens(in_sentence_tokens)))
            print("= %s"%(readable_from_tokens(out_sentence_tokens)))
            print("< %s"%readable_from_tokens(translation_tokens[:-1]))
            print("---")
        
        yield [out_sentence_tokens], translation_tokens[:-1]
        
testing_set_bleu, sentence_count = compute_bleu(text_set_translation_generator(stop_at=100, report=True), max_order=4, smooth=False)

print("Evaluated %i samples."%sentence_count)
print("BLEU: ", testing_set_bleu)

In [45]:
#Text -> text on audio data
def audio_set_text_translation_generator(stop_at = -1, report=False, model='text'):
    for in_sentence_tokens, in_prosody_tokens , out_sentence_tokens, out_prosody_tokens in audio_data_generator(AUDIO_TEST_DATA_PATH, input_lang, output_lang, n_prosody_params, input_prosody_params, output_prosody_params, dummyfy_input_prosody=True, dummyfy_output_prosody=True, stop_at=stop_at):
        if model == 'text':
            translation_tokens, _ = evaluate_text(in_sentence_tokens, input_lang, output_lang, text_encoder, text_decoder, max_seq_length)
        elif model == 'audio':
            translation_tokens, _, _, _ = evaluate_audio(in_sentence_tokens, in_prosody_tokens, input_lang, output_lang, prosodic_encoder, prosodic_decoder, max_seq_length)
        if report:
            print("> %s"%(readable_from_tokens(in_sentence_tokens)))
            print("= %s"%(readable_from_tokens(out_sentence_tokens)))
            print("< %s"%readable_from_tokens(translation_tokens[:-1]))
            print("---")
        
        yield [out_sentence_tokens], translation_tokens[:-1]
                
testing_set_bleu, sentence_count = compute_bleu(audio_set_text_translation_generator(report=False, stop_at=10, model='audio'), max_order=4, smooth=False)

print("Evaluated %i samples."%sentence_count)
print("BLEU: ", testing_set_bleu)

Evaluated 10 samples.
BLEU:  0.0


In [ ]:
#Create text translation data from compiled heroes data
output_file = "/Users/alp/Movies/heroes/transProse_data/trainable/heroes_trainable_test_text.txt"

with open(output_file, 'w') as f:
    for in_sentence_tokens, _ , out_sentence_tokens, _ in audio_data_generator(AUDIO_TEST_DATA_PATH, input_lang, output_lang, n_prosody_params, input_prosody_params, output_prosody_params):
        f.write("%s\t%s\n"%(readable_from_tokens(in_sentence_tokens), readable_from_tokens(out_sentence_tokens)))

In [63]:
t = 1.00000e-02 *torch.tensor(
       [[[ 3.7000]],

        [[ 3.7000]],

        [[ 3.7000]],

        [[ 3.1000]],

        [[ 3.7000]],

        [[ 3.7000]],

        [[ 3.7000]],

        [[ 3.7000]],

        [[ 3.7000]]])

In [76]:
a = t.squeeze().tolist()
a

[0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03099999763071537,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716]

In [79]:
#alll = []
alll += a
alll

[0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03099999763071537,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03099999763071537,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716,
 0.03700000047683716]